In [7]:
import copy
import numpy as np

class Move:
    def __init__(self):
        self.move = np.uint64(0)
        self.score = 0
        
class MoveSorter:
    def __init__(self, size = 0):
        self.size = size
        self.entries = [Move() for i in range(Position.WIDTH)]
    
    def add(self, move, score): 
        pos = self.size
        self.size += 1
        while pos and (self.entries[pos - 1].score > score):
            self.entries[pos] = self.entries[pos - 1]
            pos -= 1
        self.entries[pos].move = move;
        self.entries[pos].score = score;
        
    def getNext(self): 
        if(self.size):
            self.size -= 1
            return self.entries[self.size].move
        else:
            return 0;
    
    def reset(self):
        self.size = 0;

class Position:
    WIDTH = 7
    HEIGHT = 6
    INAROW = 4
    MIN_SCORE = -(WIDTH*HEIGHT)//2 + 3
    MAX_SCORE = (WIDTH*HEIGHT+1)//2 - 3
    
    def bottom(self, width, height):
        if width == 0:
            return np.uint64(width)
        else:
            return self.bottom(width-1, height) | np.uint64(1 << (width - 1)*(height + 1))
    def __init__(self):
        self.current_position = np.uint64(0)
        self.moves = 0
        self.mask = np.uint64(0)
        self.bottom_mask = self.bottom(Position.WIDTH, Position.HEIGHT);
        self.board_mask = self.bottom_mask * np.uint64((1 << Position.HEIGHT)-1);
    
    columnOrder = [3, 4, 2, 5, 1, 6, 0]#[7 // 2 + (1-2*(i%2))*(i+1)//2 for i in range(7)]
    
    def play(self, move):
        self.current_position ^= self.mask
        self.mask |= move
        self.moves += 1
    
    def playCol(self, column):
        self.play((self.mask + self.bottom_mask_col(column)) & self.column_mask(column));
    
    def canWinNext(self):
        return bool(self.winning_position() & self.possible())
    
    def possibleNonLosingMoves(self):
        possible_mask = self.possible()
        opponent_win = self.opponent_winning_position()
        forced_moves = possible_mask & opponent_win
        if(forced_moves):
            if(forced_moves & (forced_moves - np.uint64(1))):
                return 0;                           
            else:
                possible_mask = forced_moves
        
        return possible_mask & ~(opponent_win >> np.uint64(1))
    
    def moveScore(self, move):
        return self.popcount(self.compute_winning_position(self.current_position | move, self.mask))
    
    def popcount(self, m):
        c = 0
        while int(m):
            m &= m - np.uint64(1)
            c += 1
        return c;
    
    def winning_position(self):
        return self.compute_winning_position(self.current_position, self.mask)
    
    def opponent_winning_position(self):
        return self.compute_winning_position(self.current_position ^ self.mask, self.mask)
    
    def possible(self):
        return (self.mask + self.bottom_mask) & self.board_mask
    
    def compute_winning_position(self, position, mask):

        r = (position << np.uint64(1)) & (position << np.uint64(2)) & (position << np.uint64(3));

        p = (position << np.uint64(Position.HEIGHT+1)) & (position << np.uint64(2*(Position.HEIGHT+1)));
        r |= p & (position << np.uint64(3*(Position.HEIGHT+1)));
        r |= p & (position >> np.uint64(Position.HEIGHT+1));
        p = (position >> np.uint64(Position.HEIGHT+1)) & (position >> np.uint64(2*(Position.HEIGHT+1)));
        r |= p & (position << np.uint64(Position.HEIGHT+1));
        r |= p & (position >> np.uint64(3*(Position.HEIGHT+1)));

  
        p = (position << np.uint64(Position.HEIGHT)) & (position << np.uint64(2*Position.HEIGHT));
        r |= p & (position << np.uint64(3*Position.HEIGHT));
        r |= p & (position >> np.uint64(Position.HEIGHT));
        p = (position >> np.uint64(Position.HEIGHT)) & (position >> np.uint64(2*Position.HEIGHT));
        r |= p & (position << np.uint64(Position.HEIGHT));
        r |= p & (position >> np.uint64(3*Position.HEIGHT));

    
        p = (position << np.uint64(Position.HEIGHT+2)) & (position << np.uint64(2*(Position.HEIGHT+2)));
        r |= p & (position << np.uint64(3*(Position.HEIGHT+2)));
        r |= p & (position >> np.uint64(Position.HEIGHT+2));
        p = (position >> np.uint64(Position.HEIGHT+2)) & (position >> np.uint64(2*(Position.HEIGHT+2)));
        r |= p & (position << np.uint64(Position.HEIGHT+2));
        r |= p & (position >> np.uint64(3*(Position.HEIGHT+2)));

        return r & (self.board_mask ^ self.mask);
    
    def isWinningMove(self, column : int):
        return self.winning_position() & self.possible() & self.column_mask(column);
            
    def column_mask(self, column):
        return ((np.uint64(1) << np.uint64(Position.HEIGHT))-np.uint64(1)) << np.uint64(column*(Position.HEIGHT+1))
    
    def opponent_winning_position(self):
        return self.compute_winning_position(self.current_position ^ self.mask, self.mask)
    
    def possible(self):
        return (self.mask + self.bottom_mask) & self.board_mask;

    def key(self):
        return self.current_position + self.mask
    
    def nbMoves(self):
        return self.moves
    
    def bottom_mask_col(self, column):
        return np.uint64(1) << np.uint64(column*(Position.HEIGHT+1))


Entry = np.dtype([('key', np.uint64, 1), ('value', np.uint64, 1)])

class TransitionTable:
    def __init__(self, size):
        self.size = size
        self.T = np.full((size,), np.array((0, 0), dtype = Entry))
    def index(self, key):
        return int(key % self.size)
    def reset(self):
        del self.T
        self.T = np.full((self.size,), np.array((0, 0), dtype = Entry))
    def put(self, key, val):
        i = self.index(key)
        self.T[i]['key'] = key
        self.T[i]['value'] = val
    def get(self, key):
        i = self.index(key)
        if self.T[i]['key'] == key:
            return self.T[i]['value'] 
        else:
            return 0
    
transTable = TransitionTable(8388593)

def negamax(P, alpha, beta):
    
    next = P.possibleNonLosingMoves()
    if next == 0:
        return -(Position.WIDTH*Position.HEIGHT - P.nbMoves())//2;

    if(P.nbMoves() >= Position.WIDTH*Position.HEIGHT - 2):
        return 0; 

    Min = -(Position.WIDTH*Position.HEIGHT-2 - P.nbMoves())//2;	
    if(alpha < Min):
        alpha = Min;                   
        if(alpha >= beta):
            return alpha; 
    
    Max = (Position.WIDTH*Position.HEIGHT-1 - P.nbMoves())//2
    if val := transTable.get(P.key()):
        Max = val + Position.MIN_SCORE - 1;

    if(beta > Max):
        beta = Max;                    
        if(alpha >= beta):
            return beta;
        
    moves = MoveSorter()

    i = Position.WIDTH - 1
    while i >= 0:
        column = Position.columnOrder[i]
        columnmasked = ((np.uint64(1) << np.uint64(Position.HEIGHT))-np.uint64(1)) << np.uint64(column*(Position.HEIGHT+1))
        if move := next & columnmasked:
            moves.add(move, P.moveScore(move))
        i -= 1
  
    while(Next := moves.getNext()):
        P2 = copy.deepcopy(P)
        P2.play(Next)
        score = -negamax(P2, -beta, -alpha)
        if(score >= beta):
            return score; 
        if(score > alpha):
            alpha = score
    
    transTable.put(P.key(), alpha - Position.MIN_SCORE + 1)
    return alpha

def solve(P, weak = False): 
    if(P.canWinNext()):
        return (Position.WIDTH*Position.HEIGHT+1 - P.nbMoves())//2;
    
    Min = -(Position.WIDTH*Position.HEIGHT - P.nbMoves())//2;
    Max = (Position.WIDTH*Position.HEIGHT+1 - P.nbMoves())//2;
    if(weak):
        Min = -1;
        Max = 1;
    while Min < Max:               
        med = Min + (Max - Min)//2;
        if med <= 0 and ((Min//2) < med):
            med = Min/2;
        else:
            if med >= 0 and ((Max//2) > med):
                med = Max//2;
        r = negamax(P, med, med + 1)
        if(r <= med):
            Max = r
        else: 
            Min = r
    
    return Min

/var/folders/_j/n1j73z4j0g14gw6cprm8h7100000gn/T/ipykernel_34187/3228263498.py:147: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  Entry = np.dtype([('key', np.uint64, 1), ('value', np.uint64, 1)])


In [11]:
start = Position()
for ever in '6133112551':
    start.playCol(int(ever) - 1)
solve(start)

KeyboardInterrupt: 